In [ ]:
# プログラムを実行前に pip コマンドで以下をインストール
# (google colab の場合は以下のコメントアウトを外す)
#!pip install transformers
#!pip install mecab-python3
#!pip install fugashi[unidic-lite]
#!pip install ipadic

In [1]:
import numpy as np
from transformers import BertJapaneseTokenizer, BertForSequenceClassification, BertModel 
import torch

tokenizer = None
model = None
def init_bert():    
    '''transformerで利用するBERTモデルを初期化'''
    global tokenizer, model
    tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
    model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

def get_bert_embedding(_sentence):
    '''モデルを利用して入力文の文ベクトルを作成'''
    encoded_data = tokenizer.batch_encode_plus([_sentence], padding=True, add_special_tokens=True)
    input_ids = torch.tensor(encoded_data["input_ids"])
    with torch.no_grad():
        _outputs = model(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
    return _outputs[0][0][0].to('cpu').detach().numpy().copy()

import numpy as np
def cos_sim(x, y):
    return float(np.dot(x, y) / (np.sqrt(np.sum(x ** 2)) * np.sqrt(np.sum(y ** 2))))

In [2]:
init_bert()
print(get_bert_embedding('さいたま市は、埼玉県南部にある政令市です。'))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[ 2.91828871e-01  1.19877249e-01 -2.67125368e-01  3.53539437e-01
  3.88518780e-01 -1.80746764e-01  3.01622778e-01 -4.26075071e-01
  2.10448876e-01 -8.63645226e-04 -1.56624615e-01 -1.50796369e-01
 -3.12574565e-01  9.16761234e-02  3.26571949e-02  5.50660610e-01
 -5.07121623e-01  2.65905976e-01  2.20865190e-01  4.94762391e-01
 -2.56084740e-01 -3.03963512e-01 -4.07182544e-01 -8.60190690e-02
 -2.72209421e-02  4.03292507e-01  4.75735545e-01 -6.03722394e-01
 -3.47904563e-01  1.60953939e-01 -3.54241371e-01  3.70376706e-01
  3.34104657e-01 -5.82892835e-01 -2.48241007e-01  4.51618612e-01
 -4.67116460e-02  5.12573183e-01 -9.99632180e-02 -2.49870449e-01
  4.17204738e-01  1.53546721e-01 -3.49638134e-01  2.37890229e-01
 -5.63374400e-01 -3.17350596e-01  9.44395840e-01  4.61669505e-01
  6.07617088e-02 -5.63330293e-01  1.02728844e-01 -3.90137658e-02
  4.41129059e-02 -2.22352907e-01 -2.48581380e-01 -1.77308649e-01
 -1.45707607e-01  2.88659424e-01  3.27538639e-01 -6.76108181e-01
  4.37593833e-02 -5.11921

In [3]:
_answers = [{'ans_text': 'さいたま市は、埼玉県南部にある政令市です。', 'vector': []},
            {'ans_text': 'さいたま市は、埼玉県北部にある政令市です。', 'vector': []},
            {'ans_text': 'さいたま市は、埼玉県南部にある都市です。', 'vector': []},]

for _answer in _answers:
    _answer['vector'] = get_bert_embedding(_answer['ans_text'])

for _answer_1 in _answers:
    for _answer_2 in _answers:
        print(_answer_1['ans_text'], _answer_2['ans_text'], cos_sim(_answer_1['vector'], _answer_2['vector']))

さいたま市は、埼玉県南部にある政令市です。 さいたま市は、埼玉県南部にある政令市です。 1.0000001192092896
さいたま市は、埼玉県南部にある政令市です。 さいたま市は、埼玉県北部にある政令市です。 0.9983707666397095
さいたま市は、埼玉県南部にある政令市です。 さいたま市は、埼玉県南部にある都市です。 0.9618650078773499
さいたま市は、埼玉県北部にある政令市です。 さいたま市は、埼玉県南部にある政令市です。 0.9983707666397095
さいたま市は、埼玉県北部にある政令市です。 さいたま市は、埼玉県北部にある政令市です。 1.0000001192092896
さいたま市は、埼玉県北部にある政令市です。 さいたま市は、埼玉県南部にある都市です。 0.9613518118858337
さいたま市は、埼玉県南部にある都市です。 さいたま市は、埼玉県南部にある政令市です。 0.9618650078773499
さいたま市は、埼玉県南部にある都市です。 さいたま市は、埼玉県北部にある政令市です。 0.9613518118858337
さいたま市は、埼玉県南部にある都市です。 さいたま市は、埼玉県南部にある都市です。 1.0000001192092896
